<a href="https://colab.research.google.com/github/Annani33/SQL-for-AB-test/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL-request
https://github.com/Annani33/SQL-for-AB-test.git

In [1]:
# Import data
import pandas as pd
import numpy as np
import statsmodels.api as sm
from google.colab import drive

# Connecting Google Drive
from google.colab import drive
drive.mount("/content/drive")
# changing work folder
%cd /content/drive/MyDrive/Mate_Data_anal/Pandas/bq-results-20251016-104040-1760611268686

# Import CSV file
data = pd.read_csv("AB_test.csv")

data.head()

Mounted at /content/drive
/content/drive/MyDrive/Mate_Data_anal/Pandas/bq-results-20251016-104040-1760611268686


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Jordan,desktop,Asia,Organic Search,2,2,new account,1
1,2020-11-02,Azerbaijan,mobile,Asia,Organic Search,2,2,new account,1
2,2020-11-03,Lithuania,desktop,Europe,Social Search,2,2,new account,1
3,2020-11-03,Malta,mobile,Europe,Direct,2,1,new account,1
4,2020-11-04,North Macedonia,mobile,Europe,Organic Search,2,2,new account,1


**Group data and Calculate metrics**

In [4]:
# Group data by test and test_group
gr_data = data.groupby(["test", "test_group", "event_name"]).agg({"value": "sum"}).reset_index()

# Prepare results list
results = []
# Define metrics
metrics = {
    "add_payment_info/session": ("add_payment_info", "session"),
    "add_shipping_info/session": ("add_shipping_info", "session"),
    "begin_checkout/session": ("begin_checkout", "session"),
    "new account/session": ("new account", "session"),
}

# Loop through each test
for test in gr_data["test"].unique():
    # Filter data for each test and split it by test groups
    test_data = gr_data[gr_data["test"] == test]
    group_1 = test_data[test_data["test_group"] == 1]
    group_2 = test_data[test_data["test_group"] == 2]

    # Loop through metrics
    for metric_name, (numerator_event, denominator_event) in metrics.items():

        # 1. Data Collection
        # Group 1
        numerator_1 = group_1[group_1["event_name"] == numerator_event]["value"].sum()
        denominator_1 = group_1[group_1["event_name"] == denominator_event]["value"].sum()
        # Group 2
        numerator_2 = group_2[group_2["event_name"] == numerator_event]["value"].sum()
        denominator_2 = group_2[group_2["event_name"] == denominator_event]["value"].sum()

        # 2. Variable Initialization
        # Initialize statistical and derived results to Not a Number
        rate_A, rate_B, metric_change = np.nan, np.nan, np.nan
        z_stat, p_value = np.nan, np.nan
        significant = False
        status = "OK"

        # 3. Calculation and Testing
        # Check for valid denominators to prevent DivisionByZero and ensure meaningful test base
        if denominator_1 == 0 or denominator_2 == 0:
            status = "Error: Zero Denominator in one or both groups"
        else:
            # Calculate conversion rates
            rate_A = numerator_1 / denominator_1
            rate_B = numerator_2 / denominator_2

            # Calculate change (B vs A)
            metric_change = np.round((rate_B / rate_A - 1) * 100, 2)

            # Perform z-test for proportions
            z_stat, p_value = sm.stats.proportions_ztest(
                [numerator_1, numerator_2],
                [denominator_1, denominator_2],
                alternative="two-sided",
            )

            # Determine significance
            if p_value < 0.05:
                significant = True

        # Append results to the list
        results.append({
            "Test": test,
            "Metric": metric_name,
            "Status": status,

            "Numerator_A": numerator_1,
            "Denominator_A": denominator_1,
            "GroupA_Rate": rate_A,

            "Numerator_B": numerator_2,
            "Denominator_B": denominator_2,
            "GroupB_Rate": rate_B,

            "Metric Changes, %": metric_change,
            "Z-Stat": z_stat,
            "P-Value": p_value,
            "Significant": significant,
        })
# Final step
results_df = pd.DataFrame(results)
results_df.head(100)


,Test,Metric,Status,Numerator_A,Denominator_A,GroupA_Rate,Numerator_B,Denominator_B,GroupB_Rate,"Metric Changes, %",Z-Stat,P-Value,Significant
0,1,add_payment_info/session,OK,1988,45362,0.043825,2229,45193,0.049322,12.54,-3.924884,0.000087,True
1,1,add_shipping_info/session,OK,3034,45362,0.066884,3221,45193,0.071272,6.56,-2.603571,0.009226,True
2,1,begin_checkout/session,OK,3784,45362,0.083418,4021,45193,0.088974,6.66,-2.978783,0.002894,True
3,1,new account/session,OK,3823,45362,0.084278,3681,45193,0.081451,-3.35,1.542883,0.122859,False
4,2,add_payment_info/session,OK,2344,50637,0.046290,2409,50244,0.047946,3.58,-1.240994,0.214608,False
5,2,add_shipping_info/session,OK,3480,50637,0.068724,3510,50244,0.069859,1.65,-0.709557,0.477979,False
6,2,begin_checkout/session,OK,4262,50637,0.084168,4313,50244,0.085841,1.99,-0.952898,0.340642,False
7,2,new account/session,OK,4165,50637,0.082252,4184,50244,0.083274,1.24,-0.588793,0.556000,False
8,3,add_payment_info/session,OK,3623,70047,0.051722,3697,70439,0.052485,1.47,-0.643172,0.520112,False
9,3,add_shipping_info/session,OK,5298,70047,0.075635,5188,70439,0.073652,-2.62,1.413727,0.157442,False


**A/B Test Conclusions**

**Test #1: Successful Uplift (Implement)**
Key Outcome: Implement. Test #1 achieved statistically significant growth across three key funnel metrics:

add_payment_info: +12.54% (p=0.000087)

add_shipping_info: +6.56% (p=0.009226)

begin_checkout: +6.66% (p=0.002894)

The new_account metric showed a non-significant observed decline of −3.35%.

*Recommendation:* Implement the changes immediately. Schedule post-implementation monitoring for the new_account metric.

**Test #2: Non-Significant Result (Continue)**
Key Outcome: Non-significant. Test #2 did not show statistically significant results for any metric. The test is currently underpowered.

*Recommendation:* Extend the test for another 7 days (or until the necessary sample size is achieved) to confirm or reject the observed effect with sufficient statistical power.

**Test #3: Significant Decline (Reject)**
Key Outcome: Reject. Test #3 resulted in a statistically significant decline in the critical begin_checkout metric by −3.35% (p=0.035). All other metrics were non-significant.

*Recommendation:* Reject the changes. The recorded drop negatively impacts the checkout funnel.

**Test #4: Overall Negative Impact (Reject & Revert)**
Key Outcome: Reject and Revert. Test #4 demonstrated a statistically significant worsening for two core metrics:

begin_checkout: −2.35% (0.045934)

new_account: −3.36% (p=0.017527)

The observed (though non-significant) drops in add_payment_info and add_shipping_info support the negative trend.

*Recommendation:* Reject the tested changes and immediately revert to the original version to prevent further conversion losses.